In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

/home/crysis/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
peft_model_id = "../../llm-model/Llama2-law-finetuning-da/checkpoint-11520"

In [3]:
config = PeftConfig.from_pretrained(peft_model_id)
bnb_confg = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_confg, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 14/14 [00:06<00:00,  2.11it/s]


In [5]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_featur

In [18]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=4096,
        early_stopping=True,
        do_sample=False,
    )
    print(q)
    return tokenizer.decode(gened[0]).replace(q, "")

In [20]:
gen("보험금 지급 청구권은 상속재산에 포함되나요?")

Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: 보험금 지급 청구권은 상속재산에 포함되나요? ### Response: 


'<s> 보험금 지급 청구권은 보험금액 이상의 재산이 있는 경우에 태아, 영아, 정신적 장애 장애인, 그 밖의 사람의 권리에 의한 경우에도 지급되며, 상속재산에 포함되지 않습니다. ◇ 보험금 지급 청구권의 범위 ☞ 보험금 지급 청구권은 보험금액 이상의 재산이 있는 경우에도 지급됩니다. ☞ 보험금은 보험계약자 또는 그 밖의 사람의 채권에 의해 지급되며, 상속재산에 포함되지 않습니다. ◇ 보험금 지급 청구권의 배제 ☞ 보험금액에 따라 지급권에 대한 청구권이 발생한다고 하더라도, 다른 권리 또는 청구권의 존부로 인해 보험금 지급 청구권을 제한하는 법률의 요청을 받아들일 수 있습니다. ☞ 보험회사가 이를 이유로 보험금 지급을 거절할 수 있는 경우에는 보험금 지급 청구권의 배제를 요청해야 합니다.</s>'

### langchain

In [1]:
from dotenv import dotenv_values
from langchain.llms import HuggingFacePipeline
import os

env_vars = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = env_vars.get('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = env_vars.get('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = env_vars.get('LANGCHAIN_API_KEY')

In [8]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch

In [5]:
model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
peft_model_id = "../../llm-model/Llama2-law-finetuning-da/checkpoint-11520"

config = PeftConfig.from_pretrained(peft_model_id)
bnb_confg = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_confg, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=4096)
hf = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards: 100%|██████████| 14/14 [00:04<00:00,  3.09it/s]
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 

In [18]:
from langchain.prompts import PromptTemplate

template = """ ### Instruction: {question}
               ### Response: """
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "보험금 지급 청구권은 상속재산에 포함되나요?"

chain.invoke({"question": question})

'재산취득 후 5년이 경과하거나 상속인이 상속인 직전 1순위에 해당하는 사람이 사망한 경우에만 상속재산에 포함됩니다. ◇ 금전상 이사(금전적 이사)는 재산취득 후 5년이 경과하거나 상속인이 상속인 직전 1순위에 해당하는 사람이 사망한 경우에만 상속재산에 포함됩니다. ☞ 결격사유에 해당하는 사람에 대해서는 상속재산에 포함되지 않습니다. ☞ 피상속인의 사망으로 상속인에 대한 상속이 종료되면, 금전상 이사는 더 이상 상속인을 상속취소소송을 통해 상속취소를 할 수 있습니다. ◇ 금전상 이사는 상속재산에 단지 5년이 지나면 실효가 완료됩니다. ☞ 그러므로 5년 이상 대기되어 실효를 완성하지 못하는 것이 많은데, 이것은 상속인들의 생각을 전후하여 상속인들이 지속적으로 상속재산을 청구하는 것을 알고 있기 때문이라고 할 수 있습니다.'

In [2]:
import openai
import os

In [3]:
Audio_file = open("../../test.m4a", "rb")
transcript = openai.Audio.transcript("whisper-1", Audio_file)
voice_text = transcript['text']

APIRemovedInV1: 

You tried to access openai.Audio, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
